In [ ]:
import numpy as np
import tqdm

np.set_printoptions(edgeitems=50, linewidth=100000)

In [ ]:
file = './12.txt'
lines = open(file, 'r').readlines()
lines = [l.strip('\n') for l in lines]


In [ ]:
garden = []
garden_arr = []
vals_dict = {}
for line in lines:
    values = [n for n in line]
    for value in values:
        if value not in vals_dict:
            vals_dict[value] = len(vals_dict)+1
    nums = [vals_dict[v] for v in values]
    garden.append(values)
    garden_arr.append(nums)

garden = np.array(garden)
garden_arr = np.array(garden_arr)
print(garden.shape)
print(garden_arr.shape)

for let, num in vals_dict.items():
    print(f'let {let}={num} is present {(garden==let).sum()}={(garden_arr==num).sum()} times')

In [ ]:
import cv2
# cv2.connectedComponentsWithStats(garden=='M').astype(int)

garden = garden[:15,:15]
g = (garden=='M').astype(int)
d = np.pad(np.diff(g, axis=1),1)
print(d)
print(d.sum())
dd = np.diff(d, axis=0)
print(dd)
print(dd.sum())

In [ ]:
import cv2

def fill_gaps(ar):
    anti_arr = np.where(ar==0, 1, 0).astype(np.uint8)
    comps = cv2.connectedComponentsWithStats(anti_arr)
    filled = 0
    for lbl in range(1,comps[0]):
        points = np.argwhere(comps[1]==lbl)
        minx = min([p[0] for p in points])
        maxx = max([p[0] for p in points])
        miny = min([p[1] for p in points])
        maxy = max([p[1] for p in points])
        if minx>0 and miny>0 and maxx<(ar.shape[0]-1) and maxy<(ar.shape[1]-1):
            # print(f'filling for lbl {lbl}, pos:{[minx,maxx,miny,maxy]}')
            # print(f'filling anti ar : \n{anti_arr[minx-1:maxx+2,miny-1:maxy+2]}')
            # print(f'filling ar : \n{ar[minx-1:maxx+2,miny-1:maxy+2]}')
            ar = np.where(comps[1]==lbl, 1, ar)
            filled += (comps[1]==lbl).sum()
    return ar, filled

def compute_perimeter(ar):
    ar = np.pad(ar,1)
    # points = np.argwhere(ar==1)
    # minx = min([p[0] for p in points])
    # maxx = max([p[0] for p in points])
    # miny = min([p[1] for p in points])
    # maxy = max([p[1] for p in points])
    # fixed_ar, filled = fill_gaps(ar)
    per = 0
    per += np.abs(np.diff(ar, axis=1)).sum()
    per += np.abs(np.diff(ar, axis=0)).sum()
    # for x in range(minx-1, maxx+2):
    #     per += np.abs(np.diff(ar[x,:])).sum()
    #     # per += 2
    # for y in range(miny-1, maxy+2):
    #     per += np.abs(np.diff(ar[:,y])).sum()
    #     # per += 2
    return per

areas_dict = {}
areas = []
for num in range(1,len(vals_dict)+1):
    g = np.where(garden_arr==num, 1, 0).astype(np.uint8)
    areas_dict[num] = {}

    out = cv2.connectedComponentsWithStats(g, connectivity=4)
    for lbl in range(1,out[0]):
        ar = np.where(out[1]==lbl, 1, 0)
        area = ar.sum()
        per = compute_perimeter(ar)
        areas_dict[num][lbl] = (area, per)
        areas.append((area, per))

print(len(areas))
print(areas_dict)
res = sum([area[0]*area[1] for area in areas])
print(res)

In [ ]:
# 1477762
areas1 = [area[1] for area in areas if area[0]==1]
areas2 = [area[1] for area in areas if area[0]==2]
areas3 = [area[1] for area in areas if area[0]==3]
areas4 = [area[1] for area in areas if area[0]==4]

print(f'per area 1', np.unique(areas1, return_counts=True))
print(f'per area 2', np.unique(areas2, return_counts=True))
print(f'per area 3', np.unique(areas3, return_counts=True))
print(f'per area 4', np.unique(areas4, return_counts=True))

In [ ]:
import cv2

def compute_perimeter_lats(ar):
    ar = np.pad(ar,1)
    per_x = np.pad(np.diff(ar, axis=1),1)
    per_y = np.pad(np.diff(ar, axis=0),1)
    per = np.abs(per_x).sum() + np.abs(per_y).sum()
    lats_x = np.clip(np.diff(per_x, axis=0),0,2).sum()
    lats_y = np.clip(np.diff(per_y, axis=1),0,2).sum()
    lats = lats_x + lats_y
    assert lats_x==lats_y, f'{lats_x}!={lats_y} ar: \n{ar}, '
    return per, lats

areas_dict = {}
areas = []
for num in range(1,len(vals_dict)+1):
    g = np.where(garden_arr==num, 1, 0).astype(np.uint8)
    areas_dict[num] = {}

    out = cv2.connectedComponentsWithStats(g, connectivity=4)
    for lbl in range(1,out[0]):
        ar = np.where(out[1]==lbl, 1, 0)
        area = ar.sum()

        points = np.argwhere(ar==1)
        minx = min([p[0] for p in points])
        maxx = max([p[0] for p in points])
        miny = min([p[1] for p in points])
        maxy = max([p[1] for p in points])
        plants = ar[max(0,minx-1):min(ar.shape[0],maxx+1),max(0,miny-1):min(ar.shape[1],maxy+1)]      
        
        per, lats = compute_perimeter_lats(ar)
        areas_dict[num][lbl] = (area, per, lats, plants)
        areas.append((area, per, lats))

print(len(areas))
res_per = sum([area[0]*area[1] for area in areas])
res_lats = sum([area[0]*area[2] for area in areas])
print(res_per, res_lats)

In [ ]:
g = np.array([
    [1,1,1,1,1,1],
    [1,1,1,0,0,1],
    [1,1,1,0,0,1],
    [1,0,0,1,1,1],
    [1,0,0,1,1,1],
    [1,1,1,1,1,1],
    ])
g = np.pad(g,1)
print(g)
d = np.pad(np.diff(g, axis=1),1)
print(d)
print(np.abs(d).sum())
dd = np.diff(d, axis=0)
print(dd)
print(np.clip(dd,0,10).sum())

In [ ]:
# 1477762
areas1 = [area[1] for area in areas if area[0]==1]
areas2 = [area[1] for area in areas if area[0]==2]
areas3 = [area[1] for area in areas if area[0]==3]
areas4 = [area[1] for area in areas if area[0]==4]
areas5 = [area[1] for area in areas if area[0]==5]
areas6 = [area[1] for area in areas if area[0]==6]

print(f'per area 1', np.unique(areas1, return_counts=True))
print(f'per area 2', np.unique(areas2, return_counts=True))
print(f'per area 3', np.unique(areas3, return_counts=True))
print(f'per area 4', np.unique(areas4, return_counts=True))
print(f'per area 5', np.unique(areas5, return_counts=True))
print(f'per area 6', np.unique(areas6, return_counts=True))